In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Ridge
from scipy.sparse import hstack

1. Загрузка данных

In [2]:
train = pd.read_csv('C:/Users/1/Data/Salary-train/salary-train.csv')
test = pd.read_csv('C:/Users/1/Data/Salary-train/salary-test-mini.csv')

In [3]:
train.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355


In [4]:
test

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,We currently have a vacancy for an HR Project ...,Milton Keynes,contract,NaN
1,A Web developer opportunity has arisen with an...,Manchester,permanent,NaN


2. Предобработка тренировочных данных

In [5]:
train['FullDescription'] = train['FullDescription'].str.lower()
train['FullDescription'] = train['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)

In [6]:
# создание TF-IDF матрицы для столбца FullDescription
vectorizer = TfidfVectorizer(min_df = 5)
X_train_TF_IDF = vectorizer.fit_transform(train['FullDescription'])

In [7]:
# заполнение отсутствующих значений
train['LocationNormalized'].fillna('nan', inplace=True)
train['ContractTime'].fillna('nan', inplace=True)

In [8]:
# Использование DictVectorizer для получения one-hot encoding признаков LocationNormalized и ContractTime
# в данном случае fit_transform получает в качестве аргумента список словарей; длина списка - количество объектов
# при кодировании словари сортируются по ключам и по значениям
# 3 значения в ContractTime и 1763 значения в LocationNormalized
# если значения строкового типа, то каждый уникальный ключ-значение станет признаком для one-hot encoding 
# получаем матрицу (строки - объекты (вектор one-hot encoding), столбцы - признаки (примут значение 0 или 1)
# здесь будет 2 единицы (для значения LocationNormalized и ContractTime), остальные нули

enc = DictVectorizer() # если в качестве аргументы передать sparse=False, то после fit_transform получим ndarray
X_train_categ = enc.fit_transform(train[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [9]:
# соединяем признаки TF-IDF и one-hot encoding
X_train = hstack([X_train_TF_IDF, X_train_categ])

In [10]:
Y_train = train['SalaryNormalized'].values

3. Обучение модели

In [11]:
model = Ridge(alpha = 1.0, random_state = 241)
model.fit(X_train, Y_train)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=241, solver='auto', tol=0.001)

4. Предобработка тестовых данных

In [12]:
test['FullDescription'] = test['FullDescription'].str.lower()
test['FullDescription'] = test['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)

# # создание TF-IDF матрицы
X_test_TF_IDF = vectorizer.transform(test['FullDescription'])

test['LocationNormalized'].fillna('nan', inplace=True)
test['ContractTime'].fillna('nan', inplace=True)

# one-hot encoding
X_test_categ = enc.transform(test[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [13]:
X_test = hstack([X_test_TF_IDF, X_test_categ])

5. Предсказание

In [14]:
pred = model.predict(X_test)

In [15]:
for i in pred:
    print(round(i, 2), end = ' ')

56561.53 37143.18 